In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

from IPython.display import HTML
from tqdm import tqdm

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
path_shared = '../data/{}'
games_df = pd.read_csv(path_shared.format('games.csv'))
plays_df = pd.read_csv(path_shared.format('plays.csv'))
players_df = pd.read_csv(path_shared.format('players.csv'))

In [ ]:
out_dir_path = '../output/{}'

#First time Ops to get cache

In [ ]:
player_stats = pd.DataFrame()
passes = pd.DataFrame()
vals = []

for week in range(1, 18):
        for game_id in sorted(os.listdir(out_dir_path.format(f'{week}'))):
            if os.path.isdir(out_dir_path.format(f'{week}/{game_id}')):
                for play_id in sorted(os.listdir(out_dir_path.format(f'{week}/{game_id}'))):
                    vals.append((week, game_id, play_id))

for (w, gid, pid) in tqdm(vals):
    with open(out_dir_path.format(f'{w}/{gid}/{pid}/player_stats.pkl'), 'rb') as f:
        player_stats = player_stats.append(pd.read_pickle(f))
    with open(out_dir_path.format(f'{w}/{gid}/{pid}/passes.pkl'), 'rb') as f:
        passes = passes.append(pd.read_pickle(f))
player_stats = player_stats.reset_index(drop=True)
passes = passes.reset_index(drop=True)
# player_stats = player_stats.loc[(player_stats.frameId>=14)&(player_stats.frameId<=47)]
player_stats.head()

In [ ]:
ep1t = passes.groupby(['gameId', 'playId']).eppa1_tot.nlargest(1).droplevel(2)
ep2t = passes.groupby(['gameId', 'playId']).eppa2_tot.nlargest(1).droplevel(2)
mep1xv = passes.groupby(['gameId', 'playId']).max_eppa1_eppa1_xval.nlargest(1).droplevel(2)
mep2xv = passes.groupby(['gameId', 'playId']).max_eppa2_eppa2_xval.nlargest(1).droplevel(2)
mep2xv

In [ ]:
play_vals = pd.concat([ep1t, ep2t, mep1xv, mep2xv], axis=1).reset_index()
play_vals

In [ ]:
play_vals.to_pickle(out_dir_path.format('eppa_by_play.pkl'))

#Read Cache

In [ ]:
passes = pd.read_pickle(out_dir_path.format('all_passes.pkl'))
player_stats = pd.read_pickle(out_dir_path.format('all_player_stats.pkl'))
play_vals = pd.read_pickle(out_dir_path.format('eppa_by_play.pkl'))

In [ ]:
passes.loc[(passes.gameId==2018120212)&(passes.playId==2971)]

# Passes analysis

In [ ]:
eppas = passes.sort_values(['gameId', 'playId', 'frameId']).groupby(['gameId', 'playId']).tail(1)[['gameId', 'playId', 'frameId' ,'off_team', 'def_team','eppa1_tot','max_eppa1_eppa1_xval','true_eppa1_xval', 'xepa_inc', 'true_ppc_off', 'true_ppc_def', 'true_xepa_comp', 'true_xepa']]
eppas

In [ ]:
plays_df.columns

In [ ]:
epm = pd.merge(eppas, plays_df[['gameId', 'playId', 'epa', 'passResult']].loc[(plays_df.penaltyCodes.isnull())&((plays_df.passResult!='IN'))], how='left', on=['gameId', 'playId']).dropna()
epm

In [ ]:
from matplotlib import pyplot as plt

y1 = epmf.true_eppa1_xval.to_numpy()
x = epmf.true_xepa.to_numpy()
plt.scatter(x, y1, label=f'true xval Correlation = {np.round(np.corrcoef(x,y1)[0,1], 2)}')

# Plot
plt.xlabel('True pass eppa xval vs xepa')
plt.legend()
plt.show()

In [ ]:
x = epm.true_eppa1_xval.to_numpy()
y1 = epmf.eppa1_tot.to_numpy()
y2 = epmf.max_eppa1_eppa1_xval.to_numpy()
plt.scatter(x, y1, label=f'eppa1_tot Correlation = {np.round(np.corrcoef(x,y1)[0,1], 2)}')
plt.scatter(x, y2, label=f'eppa1_max Correlation = {np.round(np.corrcoef(x,y2)[0,1], 2)}')

# Plot
plt.xlabel('True pass xVal vs overall eppas')
plt.legend()
plt.show()

In [ ]:
x = epm.true_xepa.to_numpy()
y1 = epmf.eppa1_tot.to_numpy()
y2 = epmf.max_eppa1_eppa1_xval.to_numpy()
plt.scatter(x, y1, label=f'eppa1_tot Correlation = {np.round(np.corrcoef(x,y1)[0,1], 2)}')
plt.scatter(x, y2, label=f'eppa1_max Correlation = {np.round(np.corrcoef(x,y2)[0,1], 2)}')

# Plot
plt.xlabel('True pass xepa vs overall eppas')
plt.legend()
plt.show()

In [ ]:
x = passes.dropna().true_ppc_off.to_numpy()
y1 = passes.dropna().true_pint_off.to_numpy()
plt.scatter(x, y1, label=f'Correlation = {np.round(np.corrcoef(x,y1)[0,1], 2)}')

# Plot
plt.xlabel('Pint vs ppc')
plt.legend()
plt.show()

In [ ]:
x = passes.dropna().max_eppa1_ppc_off.to_numpy()
y1 = passes.dropna().max_eppa1_pint_off.to_numpy()
plt.scatter(x, y1, label=f'Correlation = {np.round(np.corrcoef(x,y1)[0,1], 2)}')

# Plot
plt.xlabel('Pint vs ppc')
plt.legend()
plt.show()

In [ ]:
epm.eppa1_tot.describe()

In [ ]:
epmf = epm.loc[(epm.passResult=='C')&(epm.epa<4)&(epm.epa>-2)]
x = epmf.epa.to_numpy()
y1 = epmf.true_eppa1_xval.to_numpy()
y2 = epmf.true_eppa2_xval.to_numpy()
plt.scatter(x, y1, label=f'true_eppa1_xval Correlation = {np.round(np.corrcoef(x,y1)[0,1], 2)}')
plt.scatter(x, y2, label=f'true_eppa2_xval Correlation = {np.round(np.corrcoef(x,y2)[0,1], 2)}')

# Plot
plt.title('Scatterplot and Correlations')
plt.legend()
plt.show()

In [ ]:
epm['diff'] = epm.true_eppa2_xval-epm.epa
epm.sort_values('diff')

In [ ]:
epmf = epm.loc[(epm.passResult=='I')]

x = epmf.epa.to_numpy()
y1 = epmf.xepa_inc.to_numpy()
plt.scatter(x, y1, label=f'true_eppa1_xval Correlation = {np.round(np.corrcoef(x,y1)[0,1], 2)}')

# Plot
plt.title('Scatterplot and Correlations')
plt.legend()
plt.show()

In [ ]:
_, bins = pd.cut(passes.true_trans_ppc.dropna(), 10, retbins=True, precision=10)
bins.round(5)

In [ ]:
((passes.true_ppc_off).dropna()).plot.hist()

In [ ]:
(passes.true_trans_ppc.dropna()*100000).quantile([0.001,0.005,0.01,0.05,0.1,0.2, 0.5, 0.8, 0.9, 0.95, 0.99, 0.995, 0.999])

# Player Stats

In [ ]:
((passes.true_ppc_off**1.2).dropna()).plot.hist()

In [ ]:
aggs.loc[(aggs.team_pos=='OFF')&(aggs[('ind_eppa1', 'count')]>2000)].sort_values(('ind_eppa1', 'mean'), ascending=False).round(3).head(10)